# mini-compilador (Portugol -> C)


Este projeto implementou um mini-compilador que traduz um subconjunto de Portugol para C. A ideia é percorrer as mesmas etapas estudadas em compiladores: primeiro o código fonte é quebrado em tokens (análise léxica), depois esses tokens são organizados em uma estrutura sintática (análise sintática/AST), em seguida são feitas validações semânticas com apoio de uma tabela de símbolos, e por fim o programa equivalente em C é gerado.

## Setup do ambiente

In [ ]:
import sys, os
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
from pprint import pprint


print("Faça upload do arquivo: compilador.zip")
print("Você pode encontrar as instruções para realizar tal ação em: https://github.com/Gabriel-c0Nsp/PortugolToC-compiler")
files.upload()

!unzip -o compilador.zip

sys.path.append("/content/compilador")

Faça upload do arquivo: compilador.zip
Você pode encontrar as instruções para realizar tal ação em: https://github.com/Gabriel-c0Nsp/PortugolToC-compiler


Saving compilador.zip to compilador (1).zip
Archive:  compilador.zip
  inflating: compilador/main.py      
 extracting: compilador/src/__init__.py  
  inflating: compilador/src/__pycache__/gerador_c.cpython-312.pyc  
  inflating: compilador/src/__pycache__/tabela_simbolos.cpython-312.pyc  
  inflating: compilador/src/__pycache__/erros.cpython-312.pyc  
  inflating: compilador/src/__pycache__/semantico.cpython-312.pyc  
  inflating: compilador/src/__pycache__/ast_nodes.cpython-312.pyc  
  inflating: compilador/src/__pycache__/lexer.cpython-312.pyc  
  inflating: compilador/src/__pycache__/parser.cpython-312.pyc  
  inflating: compilador/src/__pycache__/__init__.cpython-312.pyc  
  inflating: compilador/src/semantico.py  
  inflating: compilador/src/lexer.py  
  inflating: compilador/src/gerador_c.py  
  inflating: compilador/src/erros.py  
  inflating: compilador/src/tabela_simbolos.py  
  inflating: compilador/src/parser.py  
  inflating: compilador/src/ast_nodes.py  


## Análise Léxica

In [ ]:
from src.lexer import Lexer

A análise léxica é a etapa que transforma o texto do programa em uma sequência linear de tokens. Cada token representa uma unidade significativa da linguagem: palavras-chave (`inteiro`, `se`, `enquanto`), identificadores (`x`, `nome`), literais (`10`, `3.14`, `"Maria"`), operadores (`+`, `=`, `>`) e delimitadores (`;`, `(`, `)`).

O lexer deste projeto foi implementado com expressões regulares. Ele percorre o texto da esquerda para a direita e, em cada posição, tenta casar o maior token possível respeitando a prioridade das regras. Em caso de caractere inesperado, a execução para e é levantado um erro léxico contendo a posição (linha e coluna), facilitando depuração.

O resultado do lexer é uma lista de tokens, que será a entrada do parser.

## Análise Sintática

In [ ]:
from src.parser import Parser

A análise sintática verifica se a sequência de tokens segue a gramática do subconjunto de Portugol escolhido. Além de validar a estrutura, ela constrói uma AST (Abstract Syntax Tree), que é uma árvore que representa o programa de forma estruturada: declarações, comandos e expressões aparecem como nós com campos bem definidos.

Neste projeto, o parser é descendente recursivo (top-down). Isso significa que cada não-terminal da gramática é implementado como uma função (por exemplo, `comando()`, `expr()`, `termo()`, `fator()`), e o parser vai “descendo” na estrutura do programa conforme consome tokens.

## Semântica + Tabela de Símbolos

In [ ]:
from src.semantico import AnalisadorSemantico, ErroSemantico

O analisador semântico percorre a AST e mantém uma tabela de símbolos. A tabela de símbolos registra informações do programa, como:

- variáveis: nome, tipo e escopo

- rotinas: procedimentos/funções, lista de parâmetros e (para função) tipo de retorno

O projeto utiliza escopos empilhados: ao entrar em um bloco (por exemplo, corpo de se, senao, enquanto, função/procedimento), um novo escopo é criado e ao sair, ele é removido.

Além disso, o analisador semântico infere tipos de expressões e associa esses tipos aos nós da AST (por exemplo, uma expressão `a + 1` pode ser inteiro ou real dependendo dos operandos). Essa informação é usada tanto para validação quanto para escolher a forma correta de gerar C.

## Geração de C

In [ ]:
from src.gerador_c import GeradorC

A última etapa transforma a AST em código C compilável. Para cada nó do programa, há uma regra de emissão correspondente.

O código final inclui os headers necessários:

- #include <stdio.h> para printf

- #include <string.h> para strcpy ao lidar com cadeia

Mapeamentos principais:

- inteiro → int

- real → float

- cadeia → char nome[100] (array fixo, suficiente para o escopo do projeto)

Procedimentos e funções são gerados como definições C antes do main, e o restante do programa vira o corpo do main.

## Erros

In [ ]:
from src.erros import ErroCompilador

O compilador sinaliza problemas usando exceções com mensagens claras, e quando possível inclui linha e coluna do ponto onde o erro foi detectado. Na prática, os erros aparecem na execução do notebook/terminal como uma mensagem única, permitindo localizar rapidamente o problema no código de entrada.

Essa metodologia garante que o código gerado em C só é produzido quando o programa em Portugol está consistente dentro do subconjunto definido no trabalho.

## Execução dos módulos (projeto)

In [ ]:
def compilar(codigo: str, mostrar_tokens=False, mostrar_ast=False) -> str:
    try:
        tokens = Lexer().tokenizar(codigo)
        arvore = Parser(tokens).parse()

        sem = AnalisadorSemantico()
        sem.analisar(arvore)

        ger = GeradorC(sem.tabela, sem.tipos_expr)
        codigo_c = ger.gerar(arvore)
        print(codigo_c)
        return codigo_c

    except ErroCompilador as e:
        print("ERRO:", e)
        return ""


## Execução de exemplos e validação

In [ ]:
codigo = """
inteiro x;
x = 10;
escreva(x);
"""
c_gerado = compilar(codigo)


In [ ]:
codigo = """
inteiro x;
x = 10;

se (x > 10) entao
  x = x + 1;
senao
  x = 0;
fimse

escreva(x);
"""
c_gerado = compilar(codigo)

In [ ]:
codigo = """
inteiro x;
x = 0;

enquanto (x < 5) faca
  x = x + 1;
fimenquanto

escreva(x);
"""
c_gerado = compilar(codigo)


In [ ]:
codigo = """
procedimento mostrar(x)
inicio
  escreva(x);
fim

inteiro a;
a = 10;
mostrar(a);
"""
c_gerado = compilar(codigo)

In [ ]:
codigo = """
funcao soma(a, b)
inicio
  retorne a + b;
fim

inteiro x;
x = soma(2, 3);
escreva(x);
"""
c_gerado = compilar(codigo)
